In [4]:
import pandas as pd
import numpy as np

In [5]:
# Load ASCI Data
df = pd.read_csv("ACSI_Scores_Cleaned.csv")
df

,company_name,sector,publicly_traded,ticker,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,FedEx,Consumer Shipping,Yes,FDX,85,86,82,80,83,83,...,83,82,85,82,82,82,82,81,79,80
1,Atmos Energy,Investor-Owned Energy Utilities,Yes,ATO,NaN,NaN,NaN,NaN,NaN,NaN,...,81,83,85,83,82,77,80,80,78,78
2,Salt River Project,Municipal Energy Utilities,No,NaN,,,,,,,...,82,81,81,79,80,77,78,80,78,77
3,CenterPoint Energy,Investor-Owned Energy Utilities,Yes,CNP,NaN,NaN,NaN,NaN,NaN,NaN,...,80,82,84,81,81,76,79,82,80,77
4,UPS,Consumer Shipping,Yes,UPS,83,87,82,77,79,81,...,85,81,84,82,82,80,81,82,79,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,Pillsbury,Food Manufacturing,No,NaN,86,87,82,83,84,84,...,,,,,,,,,,NaN
560,Quaker (PepsiCo),Food Manufacturing,Yes,PEP,82,82,85,83,83,86,...,84,86,87,85,80,83,84,84,84,NaN
561,Ralston Foods,Food Manufacturing,No,NaN,79,81,NaN,,,,...,,,,,,,,,,NaN
562,Hillshire Brands,Food Manufacturing,No,NaN,82,84,80,80,81,82,...,82,83,84,82,NaN,,,,,NaN


In [6]:
# Filter Data
# remove companies without publicly traded or ticker data or aren't publicly traded (redundant but doing it anyway...)
df = df.dropna(subset=['ticker'])
df = df[df.publicly_traded == "Yes"]
df

,company_name,sector,publicly_traded,ticker,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,FedEx,Consumer Shipping,Yes,FDX,85,86,82,80,83,83,...,83,82,85,82,82,82,82,81,79,80
1,Atmos Energy,Investor-Owned Energy Utilities,Yes,ATO,NaN,NaN,NaN,NaN,NaN,NaN,...,81,83,85,83,82,77,80,80,78,78
3,CenterPoint Energy,Investor-Owned Energy Utilities,Yes,CNP,NaN,NaN,NaN,NaN,NaN,NaN,...,80,82,84,81,81,76,79,82,80,77
4,UPS,Consumer Shipping,Yes,UPS,83,87,82,77,79,81,...,85,81,84,82,82,80,81,82,79,76
5,NextEra Energy,Investor-Owned Energy Utilities,Yes,NEE,77,74,69,75,74,76,...,78,80,80,76,77,76,75,76,77,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,Hershey,Food Manufacturing,Yes,HSY,88,88,84,84,86,85,...,84,85,86,86,79,84,86,86,84,NaN
554,Kellogg,Food Manufacturing,Yes,K,84,85,81,83,81,83,...,80,83,85,81,80,81,81,81,80,NaN
555,Kraft Heinz,Food Manufacturing,Yes,KHC,84,85,82,84,83,82,...,83,81,86,84,79,83,82,82,81,NaN
558,Nestle (Ferrero),Food Manufacturing,Yes,NSRGF,86,82,83,83,81,84,...,84,85,83,85,78,84,82,82,81,NaN


In [8]:
import yfinance as yf

# define function for fetching stock price data for the companies and years we have consumer satisfaction data for 

def get_stock_prices(tickers, years):
    stock_prices = pd.DataFrame()
    for year in years:
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        for ticker in tickers:
            try:
                stock_info = yf.download(ticker, start=start_date, end=end_date)
                stock_price = stock_info['Open'][0] # get the opening price for the first day of the year
                stock_prices.loc[ticker, str(year)] = stock_price # add the price to the DataFrame
            except:
                print(f"Error downloading data for {ticker}")
    return stock_prices

def get_stock_prices_monthly(tickers, years):
    #similar function but for fetching monthly data instead of annual data
    stock_prices = pd.DataFrame()
    for year in years:
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        for ticker in tickers:
            try:
                stock_info = yf.download(ticker, start=start_date, end=end_date, interval='1mo')
                stock_price = stock_info['Open'].iloc[0] # get the opening price for the first day of the month
                stock_prices.loc[ticker, str(year)] = stock_price # add the price to the DataFrame
            except:
                print(f"Error downloading data for {ticker}")
    return stock_prices

In [19]:
stock_tickers = list(df['ticker'])

# download annual stock data
stock_df = get_stock_prices(
    tickers = list(df['ticker']),
    years = [year for year in range(1995, 2022)]
    )

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- UPS: Data doesn't exist for startDate = 788936400, endDate = 820386000
Error downloading data for UPS
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SRE: Data doesn't exist for startDate = 788936400, endDate = 820386000
Error downloading data for SRE
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 com

In [20]:

# download annual stock data
sp_500_stock_df = get_stock_prices(
    tickers = ['SPY'],
    years = [year for year in range(1995, 2022)]
    )

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [22]:
sp_500_stock_df.to_csv("sp_500_stock_df.csv")

In [12]:
# save the stock data to .csv file
stock_df.to_csv("stock_data.csv")

In [17]:
# save the score data to .csv file
df.to_csv("score_data.csv")